In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import cv2

import glob
import os
import boto3
import tempfile
from sqlalchemy import create_engine, MetaData, Table, select, and_, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from scipy.stats import norm
import tqdm
import pickle
from itertools import combinations
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.accuracy_metrics import AccuracyMetricsGenerator
from aquabyte.optics import euclidean_distance

from PIL import Image, ImageDraw
from multiprocessing import Pool, Manager
import copy
import uuid
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_rows', 500)





In [ ]:
research_sql_credentials = json.load(open(os.environ["SQL_CREDENTIALS"]))
research_rds_access_utils = RDSAccessUtils(research_sql_credentials)
sql_engine = research_rds_access_utils.sql_engine
Session = sessionmaker(bind=sql_engine)
session = Session()

Base = automap_base()
Base.prepare(sql_engine, reflect=True)
Enclosure = Base.classes.enclosures
Calibration = Base.classes.calibrations
GtsfDataCollection = Base.classes.gtsf_data_collections
StereoFramePair = Base.classes.stereo_frame_pairs

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')


In [ ]:
session.rollback()

In [ ]:
df = pd.read_hdf('/root/data/temp/results_ae9e2ce03f6417710870aefeb04b543ffc513eb8_gtsf_data_bati_axiom_rectified.h5', 'table')





In [ ]:
session.rollback()

augmented_df = pd.DataFrame()
prefix = 'https://s3-eu-west-1.amazonaws.com/'
bucket = 'aquabyte-frames-resized-inbound'
count = 0
for idx, row in df.iterrows():
#     if count % 100 == 0:
#         print('hi') 
    print(count)
    count += 1
    left_image_url = row.left_image_url
    key = os.path.join(os.path.dirname('/'.join(left_image_url.replace(prefix, '').split('/')[1:])), 'capture.json')
    capture_f = s3_access_utils.download_from_s3(bucket, key)
    capture_metadata = json.load(open(capture_f))
    gtsf_fish_identifier = capture_metadata['gtsf_fish_identifier']
    gtsf_data_collection = session.query(GtsfDataCollection).filter(GtsfDataCollection.gtsf_fish_identifier == gtsf_fish_identifier).all()[0]
    ground_truth_metadata = json.loads(gtsf_data_collection.ground_truth_metadata)
    weight = ground_truth_metadata['data']['weight']
    df_row = dict(row)
    df_row['gtsf_fish_identifier'] = gtsf_fish_identifier
    df_row['weight'] = weight
    augmented_df = augmented_df.append(df_row, ignore_index=True)
#     except ValueError:
#         print('Error with idx = {}'.format(idx))
    
        


In [ ]:
augmented_df[augmented_df.gtsf_fish_identifier=='190301010001']

In [ ]:
plt.figure(figsize=(20, 10))
plt.scatter(augmented_df.weight, augmented_df.estimated_biomass_g)
plt.plot([0, 10000], [0, 10000])
plt.ylim([0, 10000])
plt.grid()
plt.show()

In [ ]:
np.abs((augmented_df.estimated_biomass_g - augmented_df.weight)/augmented_df.weight).median()

In [ ]:
import numpy as np
import json
import datetime
from sqlalchemy import create_engine, MetaData, Table, exc, exists, select, literal
import pickle

def euclidean_distance(p1, p2):
    return ((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2)**0.5


def convert_to_world_point(x, y, d, parameters):
    """ from pixel coordinates to world coordinates """
    # get relevant parameters
    pixel_count_height = parameters["pixelCountHeight"]
    pixel_count_width = parameters["pixelCountWidth"]
    sensor_width = parameters["imageSensorWidth"]
    sensor_height = parameters["imageSensorHeight"]
    focal_length = parameters["focalLength"]

    image_center_x = pixel_count_height / 2.0
    image_center_y = pixel_count_width / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (sensor_height / pixel_count_height)
    sensor_z = px_z * (sensor_width / pixel_count_width)

    # now move to world coordinates
    world_y = d
    world_x = (world_y * sensor_x) / focal_length
    world_z = (world_y * sensor_z) / focal_length
    return np.array([world_x, world_y, world_z])


def depth_from_disp(disp, parameters):
    """ calculate the depth of the point based on the disparity value """
    focal_length_pixel = parameters["focalLengthPixel"]

    baseline = parameters["baseline"]
    depth = focal_length_pixel * baseline / np.array(disp)
    return depth


def pixel2world(left_crop, right_crop, parameters):
    """2D pixel coordinates to 3D world coordinates"""

    # first create a dic with crop keypoints
    image_coordinates = {"leftCrop": {},
                         "rightCrop": {}}
    for keypoint in left_crop:
        name = keypoint["keypointType"]
        image_coordinates["leftCrop"][name] = [keypoint["xFrame"], keypoint["yFrame"]]
    for keypoint in right_crop:
        name = keypoint["keypointType"]
        image_coordinates["rightCrop"][name] = [keypoint["xFrame"], keypoint["yFrame"]]

    # then loop through the right crop keypoints and calculate the world coordinates
    world_coordinates = {}
    for keypoint in left_crop:
        name = keypoint["keypointType"]
        disparity = image_coordinates["leftCrop"][name][0] - image_coordinates["rightCrop"][name][0]
        depth = depth_from_disp(disparity, parameters)
        world_point = convert_to_world_point(image_coordinates["leftCrop"][name][1],
                                             image_coordinates["leftCrop"][name][0],
                                             depth,
                                             parameters)
        world_coordinates[name] = world_point
    return world_coordinates


def coord2biomass(world_keypoints, model):
    """from coordinates to biomass"""

    mean = model['mean']
    std= model['std']
    PCA_components = model['PCA_components']
    reg_coef = model['reg_coef']
    reg_intercept = model['reg_intercept']
    body_parts = model['body_parts']
    print(body_parts)
    # calculate pairwise distances for production coord
    # based on the exact ordering reflected in the body_parts
    # variable above

    pairwise_distances = []
    for i in range(len(body_parts)-1):
        for j in range(i+1, len(body_parts)):
            dist = euclidean_distance(world_keypoints[body_parts[i]], world_keypoints[body_parts[j]])
            pairwise_distances.append(dist)

    interaction_values_quadratic = []
    for i in range(len(pairwise_distances)):
        for j in range(i, len(pairwise_distances)):
            dist1 = pairwise_distances[i]
            dist2 = pairwise_distances[j]
            interaction_values_quadratic.append(dist1 * dist2)

    interaction_values_cubic = []
    for i in range(len(pairwise_distances)):
        for j in range(i, len(pairwise_distances)):
            for k in range(j, len(pairwise_distances)):
                dist1 = pairwise_distances[i]
                dist2 = pairwise_distances[j]
                dist3 = pairwise_distances[k]
                interaction_values_cubic.append(dist1 * dist2 * dist3)


    X = np.array(pairwise_distances + interaction_values_quadratic + interaction_values_cubic)

    X_normalized = (X - model['mean']) / model['std']
    X_transformed = np.dot(X_normalized, model['PCA_components'].T)
    prediction = np.dot(X_transformed, reg_coef) + reg_intercept
    return prediction


In [ ]:
model = pickle.load(open('/root/data/models/new_model.pkl', 'rb'))
keypoints = df[df.id == 506696.0].keypoints.iloc[0]
left_crop = keypoints['leftCrop']
right_crop = keypoints['rightCrop']
cm = keypoints['cameraParameters']
coordinates = pixel2world(left_crop, right_crop, cm)
coord2biomass(coordinates, model)

In [ ]:
{"version": 2, "leftCrop": [{"xCrop": 46, "yCrop": 446, "xFrame": 879, "yFrame": 1441, "keypointType": "UPPER_LIP"}, {"xCrop": 179, "yCrop": 426, "xFrame": 1012, "yFrame": 1421, "keypointType": "EYE"}, {"xCrop": 1219, "yCrop": 73, "xFrame": 2052, "yFrame": 1068, "keypointType": "DORSAL_FIN"}, {"xCrop": 2280, "yCrop": 342, "xFrame": 3113, "yFrame": 1337, "keypointType": "ADIPOSE_FIN"}, {"xCrop": 2601, "yCrop": 482, "xFrame": 3434, "yFrame": 1477, "keypointType": "UPPER_PRECAUDAL_PIT"}, {"xCrop": 2795, "yCrop": 638, "xFrame": 3628, "yFrame": 1633, "keypointType": "HYPURAL_PLATE"}, {"xCrop": 2918, "yCrop": 677, "xFrame": 3751, "yFrame": 1672, "keypointType": "TAIL_NOTCH"}, {"xCrop": 2558, "yCrop": 718, "xFrame": 3391, "yFrame": 1713, "keypointType": "LOWER_PRECAUDAL_PIT"}, {"xCrop": 2064, "yCrop": 833, "xFrame": 2897, "yFrame": 1828, "keypointType": "ANAL_FIN"}, {"xCrop": 1490, "yCrop": 855, "xFrame": 2323, "yFrame": 1850, "keypointType": "PELVIC_FIN"}, {"xCrop": 469, "yCrop": 672, "xFrame": 1302, "yFrame": 1667, "keypointType": "PECTORAL_FIN"}], "rightCrop": [{"xCrop": 37, "yCrop": 468, "xFrame": 417, "yFrame": 1435, "keypointType": "UPPER_LIP"}, {"xCrop": 144, "yCrop": 454, "xFrame": 524, "yFrame": 1421, "keypointType": "EYE"}, {"xCrop": 1209, "yCrop": 101, "xFrame": 1589, "yFrame": 1068, "keypointType": "DORSAL_FIN"}, {"xCrop": 2291, "yCrop": 373, "xFrame": 2671, "yFrame": 1340, "keypointType": "ADIPOSE_FIN"}, {"xCrop": 2610, "yCrop": 514, "xFrame": 2990, "yFrame": 1481, "keypointType": "UPPER_PRECAUDAL_PIT"}, {"xCrop": 2810, "yCrop": 667, "xFrame": 3190, "yFrame": 1634, "keypointType": "HYPURAL_PLATE"}, {"xCrop": 2941, "yCrop": 700, "xFrame": 3321, "yFrame": 1667, "keypointType": "TAIL_NOTCH"}, {"xCrop": 2569, "yCrop": 742, "xFrame": 2949, "yFrame": 1709, "keypointType": "LOWER_PRECAUDAL_PIT"}, {"xCrop": 2061, "yCrop": 861, "xFrame": 2441, "yFrame": 1828, "keypointType": "ANAL_FIN"}, {"xCrop": 1463, "yCrop": 883, "xFrame": 1843, "yFrame": 1850, "keypointType": "PELVIC_FIN"}, {"xCrop": 417, "yCrop": 700, "xFrame": 797, "yFrame": 1667, "keypointType": "PECTORAL_FIN"}]}

In [ ]:
right_crop

In [ ]:
coordinates

In [ ]:
wkps = df[df.id == 506696.0].keypoints.iloc[0]

In [ ]:
df[df.id == 506696.0].keypoints.iloc[0]

In [ ]:
model

In [ ]:
augmented_df.shape

In [ ]:
augmented_df[augmented_df.weight < 500].gtsf_fish_identifier